In [ ]:
update = False

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh -y

# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

# CheckPoint

In [ ]:
# Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://huggingface.co/Lightricks/LTX-Video/resolve/main/ltx-video-2b-v0.9.safetensors'
model_name = 'ltx-video-2b-v0.9.safetensors'

%cd $checkpoints
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Clip

In [ ]:
# Install a Clip in permanent storage
model_url = 'https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors'
model_name = 't5xxl_fp16.safetensors'

%cd /kaggle/working/ComfyUI/models/clip
#!wget -O 'model.safetensors' 'https://civitai.com/api/download/models/198530?type=Model&format=SafeTensor&size=full&fp=fp16'
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Launch with Pinggy

In [ ]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()